In [1]:
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi
import json
import pandas as pd
import os
import datetime
import numpy as np
import glob
import sqlalchemy

dict_config=json.load(open("/mnt/clients/juba/hqjubaapp02/jliang/Projects/Big_Lots/Predictive_Model/Model_Scripts/config.json"))
high_date=dict_config['pos_end_date']
username=dict_config['username']
password=dict_config['password']
database=dict_config['database']
BL_engine=sqlalchemy.create_engine("mysql+pymysql://%s:%s@localhost/%s" % (username, password, database))
output_folder=dict_config['model_output_folder']


BigLotsAccount_id="act_271491453638620"
BigLotsDcart10_id="act_2449979771956630"

list_account_id=[BigLotsAccount_id,BigLotsDcart10_id]
# jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token=json.load(open("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/FB_token/lasest_FB_token_jian.json","r"))['latest_FB_token']



In [3]:
high_date="2020-12-19" # The first audience output week

df_id_email=pd.read_sql("select customer_id_hashed,email_address_hash from BL_Rewards_Master order by sign_up_date desc;",con=BL_engine)
df_id_email=df_id_email.drop_duplicates("customer_id_hashed")


In [4]:
# create new adueince and add the new users to the audiecne, 1 step
# Please note the change of the CustomAudience.Schema (row 41 based on the user list format)
def creatfbaudience(AudienceName,
                    AudienceDescription,
                    df,
                    ACCESS_TOKEN,
                    accountid,
                    df_use_col_name,
                    fb_schema=CustomAudience.Schema.email_hash):
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v8.0')
    
    '''
    AudienceName: str, the name shown on FB
    AudienceDescription: str, the description shown on FB
    df: pd.Dataframe, input with the identifier col
    ACCESS_TOKEN: str, access to FB marketing api
    accountid: str, account to uplaod
    df_use_col_name: str, col name in the df
    fb_schema: available schema attribution in the CustomAudience.Schema class. Default for us CustomAudience.Schema.email_hash  
    https://github.com/facebook/facebook-python-business-sdk/blob/ac390147132101763a99a917bb59bdc8a374a7ce/facebook_business/adobjects/helpers/customaudiencemixin.py
    '''
    
    myAccount = AdAccount(accountid)
    fields = [
    ]
    params = {
      'name': AudienceName,
      'subtype': 'CUSTOM',
      'description': AudienceDescription,
      'customer_file_source': 'PARTNER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    print("Audience ID:",fbname)
    audience = CustomAudience(fbname)
    users = list(df[df_use_col_name])
    listlen = len(users)
    chunck_size=50000
    count_uploaded=0
    for i in range(int(np.ceil(listlen/chunck_size))):
        starti = i*50000
        if (i+1)*50000<listlen:
            endi = (i+1)*50000
        else:
            endi = listlen
        list0 = users[starti:endi]
        
        count_uploaded+=len(list0)
        audience.add_users(fb_schema, list0)
        #print len(list0)   
    print(datetime.datetime.now(),AudienceName,"uploaded",count_uploaded)


In [17]:
list_all_output_weekly_folder=os.listdir(output_folder)
folder_weekly_output=[x for x in list_all_output_weekly_folder if "output_LastWeek_NoDCM_%s"%high_date in x]
if len(folder_weekly_output)==1:
    folder_weekly_output=output_folder+folder_weekly_output[0]
    list_weekly_output=glob.glob(folder_weekly_output+"/*.csv")
else:
    raise ValueError("weekly output folder count is not 1")


In [18]:
def find_cutoff_from_outputs(key_df_type,weekly_output_folder):
    # key_df_type: "trans_1_only" or "trans_2_plus" in the keys of dict_df_type
    list_all_weekly_summary=glob.glob(weekly_output_folder+"/*.xlsx")
    list_type_excel=[x for x in list_all_weekly_summary if key_df_type in x]
    if len(list_type_excel)==1:
        file=list_type_excel[0]
    else:
        raise ValueError("excel output file count is not 1")
    excel=pd.ExcelFile(file)
    best_cutoff=excel.parse("select_score_matrix")['index'][0]
    return best_cutoff

In [19]:
def load_targeting_df(key_df_type,weekly_output_folder):
    # The targeting label is already wrote in the csv file
    # If new changes needed, we can overwrite based on the y_hat value
    list_weekly_output=glob.glob(weekly_output_folder+"/*.csv")
    file_id_list=[x for x in list_weekly_output if "df_target_ids_labeled_20" in x and key_df_type in x]
    if len(file_id_list)==1:
        file_id_list=file_id_list[0]
    else:
        raise ValueError("file_id_list file count is not 1 for %s"%key_df_type)
    df=pd.read_csv(file_id_list)
    df=df[(df['selection_label']=="target") | (df['sign_up_label']=="new_signs") | (df['email_subscription_label']=="unsub")]
    return df


    

In [20]:
# Initiate trans_1_only
key_df_type="trans_1_only"
weekly_output_folder=folder_weekly_output

for fb_account_id in list_account_id:
    audience_name = "Rewards_Predictive_Buyers_%s"%key_df_type
    description="The upcoming 3 weeks buyers from %s predictive model"%key_df_type
    
    df = load_targeting_df(key_df_type,weekly_output_folder)
    print("total rows of target selected with overwrite: %i"%df.shape[0])
    df=pd.merge(df,df_id_email,on="customer_id_hashed",how="left")
    df=df[pd.notnull(df['customer_id_hashed'])]
    df=df.drop_duplicates("customer_id_hashed")
    df=df[df['customer_id_hashed'].apply(len)==64]
    print("total unique emails of target selected with overwrite: %i"%df.shape[0])
    
    creatfbaudience(AudienceName="aud_"+audience_name,
                    AudienceDescription=description,
                    df=df,
                    df_use_col_name='email_address_hash',
                    ACCESS_TOKEN=jian_token,
                    accountid=fb_account_id)

total rows of target selected with overwrite: 1814996
total unique emails of target selected with overwrite: 1814996
Audience ID: 23846273583640265
2021-01-04 22:32:35.399720 aud_Rewards_Predictive_Buyers_trans_1_only uploaded 1814996
total rows of target selected with overwrite: 1814996
total unique emails of target selected with overwrite: 1814996
Audience ID: 23846635244320632
2021-01-04 22:35:41.026008 aud_Rewards_Predictive_Buyers_trans_1_only uploaded 1814996


In [21]:
# Initiate trans_2_plus
key_df_type="trans_2_plus"
weekly_output_folder=folder_weekly_output

for fb_account_id in list_account_id:
    audience_name = "Rewards_Predictive_Buyers_%s"%key_df_type
    description="The upcoming 3 weeks buyers from %s predictive model"%key_df_type
    
    df = load_targeting_df(key_df_type,weekly_output_folder)
    print("total rows of target selected with overwrite: %i"%df.shape[0])
    df=pd.merge(df,df_id_email,on="customer_id_hashed",how="left")
    df=df[pd.notnull(df['customer_id_hashed'])]
    df=df.drop_duplicates("customer_id_hashed")
    df=df[df['customer_id_hashed'].apply(len)==64]
    print("total unique emails of target selected with overwrite: %i"%df.shape[0])
    
    creatfbaudience(AudienceName="aud_"+audience_name,
                    AudienceDescription=description,
                    df=df,
                    df_use_col_name='email_address_hash',
                    ACCESS_TOKEN=jian_token,
                    accountid=fb_account_id)

total rows of target selected with overwrite: 10234701
total unique emails of target selected with overwrite: 10234701
Audience ID: 23846273655130265
2021-01-04 22:48:00.822879 aud_Rewards_Predictive_Buyers_trans_2_plus uploaded 10234701
total rows of target selected with overwrite: 10234701
total unique emails of target selected with overwrite: 10234701
Audience ID: 23846635282950632
2021-01-04 23:01:18.055142 aud_Rewards_Predictive_Buyers_trans_2_plus uploaded 10234701
